In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from typing import List
from itertools import permutations
data = pd.read_excel("../data/1.xlsx")

# 问题背景

- 一名员工加入后工作 3 至 15 天

- 每个车间人数为 5 至 15 人不等

某车间的员工集合随着时间的变化动态改变

| 员工数量 |        运输专班安排        |
| :------: | :------------------------: |
|   5-7    |           班次A            |
|   8-10   |        班次A、班次B        |
|  11-12   |    班次A、班次B、班次C     |
|  13-15   | 班次A、班次B、班次C、班次D |

# 硬约束条件（必须满足）

- 每个专班需安排两名员工,其中至少一名需具有驾驶资格。

- 每名员工每天至多安排一次专班。

# 公平性条件（尽可能满足）

- 使每位员工当前累计安排专班次数与当前工作天数之比（定义为公平指数）和所有员工公平指数的平均值的差值不超过0.1 

- 具体实现上体现为优先选择公平指数低的员工进行工作

# 软约束条件（在满足公平的前提下尽可能满足）

- 每个班次至少有一名已有专班经验的员工。（权重0.3）

- 每名员工在不同天内参与的专班在具体班次上应尽可能平均。（权重0.2）

- 历史事故次数较高的员工参与专班时须与历史事故次数较低的员工组成专班。历史事故次数为 0 或 1 的可认为历史事故次数较低,历史事故次数大于 3 的可认为历史事故次数较高。（权重0.1）
 

# 单天模型
## 输入

一个当前车间员工的集合，以 list 形式存储。

每一名员工拥有以下属性：

1. 员工编号

2. 总工作天数

3. 已工作天数（排版当天不计入已工作天数，尚未入职用+n表示之后n天入职）

4. 是否有驾驶资格

5. 历史事故次数

6. 安排A班次数

7. 安排B班次数

8. 安排C班次数

9. 安排D班次数

10. 公平指数（见上文定义）

In [ ]:
class member:
    def __init__(self, 
                mid, total_day, past_day, driver, accident_times, 
                A_times, B_times, C_times, D_times):
        self.mid = mid
        self.total_day = total_day
        self.past_day = past_day
        self.driver = driver
        self.accident_times = accident_times
        self.A_times = A_times
        self.B_times = B_times
        self.C_times = C_times
        self.D_times = D_times
        self.fair_ratio = 0
        self.exprience = (A_times or B_times or C_times or D_times)
    
    # calculate fair_ratio
    def cal_fair_ratio(self):
        self.fair_ratio = (self.A_times + self.B_times + self.C_times + self.D_times) / self.past_day
        if(self.past_day < 0):
            self.fair_ratio += 2
    # override < operator to sort 
    def __lt__(self, other):
        return self.fair_ratio < other.fair_ratio     

## 输出
一个排版情况，以 dictionary 形式存储，key为每个员工的编号，value 为员工排班的班次，班次为 0 表示未排班

In [ ]:
# example : {1: 'A', 2: 'B', 3:0, 4:'A', 5:'B', 6:0...}

## 算法描述
### 初始化数据
从给定的 excel 表格中读入， 初始化成员类并放入list作为算法的输入

In [ ]:
member_list = []

for i in range(data.shape[0]):
    new_member = member(
        data.loc[:,"编号"][i], 
        data.loc[:,"工作天数"][i],
        data.loc[:,"已工作天数"][i],
        data.loc[:,"驾驶资格"][i],
        data.loc[:,"历史事故"][i],
        data.loc[:,"已安排A"][i],
        data.loc[:,"已安排B"][i],
        data.loc[:,"已安排C"][i],
        data.loc[:,"已安排D"][i]
    )
    new_member.cal_fair_ratio()
    member_list.append(new_member)


## 选择备选员工
将员工列表按照公平指数从小到大排序，并且将尚未入职的员工排在最后。
计算当前需要的专班数量 n，并根据专班数量，选取不少于 n 名驾驶员，共计选取不少于 2n 名员工进入备选区。
为保证公平，优先选取公平指数小的员工。
如果在选取 n 名驾驶员和 2n 名员工的条件达到后，剩余未被选择的员工中公平指数与备选区中公平指数最大的员工相差小于等于 0.1 的部分也进入备选区
反之则只选取 n 名驾驶员和 2n 名员工的条件达到时进入备选区的员工

In [ ]:
member_list = sorted(member_list)

# caculate amount of member who is in the company already
active_member_num = len([each.past_day for each in member_list if each.past_day >= 0])

def calculate_work_num(x):
    if(x <= 7 ):
        return 1
    elif(x <= 10):
        return 2
    elif(x<= 12):
        return 3
    elif(x <= 15):
        return 4
# caculate the amount of work 
work_num = calculate_work_num(active_member_num)

alternative_list = []

def get_alternative(arr, total_num):
    cur_driver = 0
    cur_total = 0
    max_fair_ratio = 0.0
    res_list = []
    for each in arr:
        if(each.driver):
            cur_driver += 1
        cur_total += 1
        res_list.append(each)
        if(cur_driver >= total_num and cur_total >= 2*total_num):
            max_fair_ratio = each.fair_ratio
            break
    # get all alternative which meet the hard condition
        
    if(cur_driver < total_num or cur_total < 2*total_num):
        return []
    # cannot satisfy the hard condition

    for each in arr:
        if(each not in res_list):
            if(each.fair_ratio <= max_fair_ratio + 0.1):
                res_list.append(each)
    return res_list

alternative_list = get_alternative(member_list, work_num)

## 最优解选择策略
对于备选区内的员工枚举可行解。

枚举可行解时，实际上是从所有员工中选择一个 2*班次 的排列，排列中，前两位员工为A班次，第三位和第四位员工为B班次，以此类推。

对于每一种可行解，按照是否满足每个软条件进行判断并求加权和。
三个软条件的权重分别是 0.3 0.2 0.1
最后将软条件加权和最大的可行解作为最优解

In [ ]:
# arr is a possible_solution a total_num is work number
def cal_optimize_ratio(arr:List[member], total_num:int):
    opt_ratio = 0.3
    # soft condition 1
    for i in range(total_num):
        if(not arr[i*2].exprience and not arr[i*2+1].exprience):
            opt_ratio -= 0.3
            break
    
    # soft condition 3
    opt_ratio += 0.1
    for i in range(total_num):
        if(max(arr[i*2].accident_times, arr[i*2+1].accident_times) > 3):
            if(min(arr[i*2].accident_times, arr[i*2+1].accident_times) > 1 ):
                opt_ratio -= 0.1
                break
    
    # soft condition 2
    def cal_member_work_avg(mem:member, type:int):
        A_times = mem.A_times + (type == 1)
        B_times = mem.B_times + (type == 1)
        C_times = mem.C_times + (type == 1)
        D_times = mem.D_times + (type == 1)

        _tmp_list = []
        _tmp_list.append(A_times)
        if(B_times):
            _tmp_list.append(B_times)
        if(C_times):
            _tmp_list.append(C_times)
        if(C_times):
            _tmp_list.append(C_times)

        return np.std(_tmp_list)

    _condition2_list = []
    for i in range(total_num):
        _condition2_list.append(cal_member_work_avg(arr[i], i))
    opt_ratio += np.mean(_condition2_list)
    print(opt_ratio)
    return opt_ratio


ans = []
max_ratio = 0.0

for possible_solution in list(permutations(alternative_list, work_num * 2)):
    validator = True
    for i in range(work_num):
        if((not alternative_list[i*2].driver)and (not alternative_list[i*2+1].driver)):
            validator = False
            break
    if(not validator):
        continue
    
    _tmp_ratio = cal_optimize_ratio(possible_solution, work_num)
    if(_tmp_ratio > max_ratio):
        max_ratio = _tmp_ratio
        ans = possible_solution

# 公平性定义
对于不同的公平性定义，仅需修改 member 类的 `cal_fair_ratio`即可

In [ ]:
# 人力资源选择
对于给定的候选者，加入初始员工队列，如果加入后的最优解发生改变，则该员工更有价值